# -------------------- Projeto Cetus ------------------------------------

### Stage 01: Bolsa Família - Pagamentos


### Parametros

In [8]:
datref = 201601
path_lake = 'projeto-bigdata-cetus/datalake/BeneficiosCidadaos/'
nm_file_lake = 'BC_Lake_BolFamPag_'

path_stage01 = 'projeto-bigdata-cetus/stage01/BolsaFamilia/'
nm_file_stg01 = 'BC_Stage01_BolFamPag_'

### Lendo Datalake (arquivo csv)

In [6]:
rdd_00 = spark.read.format(
   "com.databricks.spark.csv").option(
    "encoding", "ISO-8859-1").option(
   "header", "true").option(
   "inferSchema", "true").option(
   "delimiter", ',').load(    
   's3a://' + path_lake + nm_file_lake + str(datref) + '.csv')
rdd_00.count()   

14020581

### Extraindo e Transformando Dados

In [41]:
rdd_01 = rdd_00.withColumnRenamed("Ano/Mês Referência","DT_DATREF")\
        .withColumnRenamed("Ano/Mês Competência","DT_DATCOMPT")\
        .withColumnRenamed("UF","ID_UF")\
        .withColumnRenamed("Código Município SIAFI","ID_COD_MUN_SIAFI")\
        .withColumnRenamed("Nome Município SIAFI","ID_NOM_MUN_SIAFI")\
        .withColumnRenamed("NIS Beneficiário","ID_NIS_BENEF")\
        .withColumnRenamed("Nome Beneficiário","ID_NOM_BENEF")\
        .withColumnRenamed("Valor Benefício","VL_VLR_PARCELA")

In [38]:
rdd_01.printSchema()

root
 |-- DT_DATREF: integer (nullable = true)
 |-- DT_DATCOMPT: integer (nullable = true)
 |-- ID_UF: string (nullable = true)
 |-- ID_COD_MUN_SIAFI: integer (nullable = true)
 |-- ID_NOM_MUN_SIAFI: string (nullable = true)
 |-- ID_NIS_BENEF: long (nullable = true)
 |-- ID_NOM_BENEF: string (nullable = true)
 |-- VL_VLR_PARCELA: string (nullable = true)
 |-- DT_INFO: integer (nullable = true)
 |-- ID_PROC: string (nullable = true)

### Alterando separador decimal de , para .

In [51]:
from pyspark.sql.functions import regexp_replace, col
rdd_01 = rdd_01.withColumn('VL_VLR_PARCELA', regexp_replace('VL_VLR_PARCELA', ',', '.').cast('float'))
rdd_01.printSchema()

root
 |-- DT_DATREF: integer (nullable = true)
 |-- DT_DATCOMPT: integer (nullable = true)
 |-- ID_UF: string (nullable = true)
 |-- ID_COD_MUN_SIAFI: integer (nullable = true)
 |-- ID_NOM_MUN_SIAFI: string (nullable = true)
 |-- ID_NIS_BENEF: long (nullable = true)
 |-- ID_NOM_BENEF: string (nullable = true)
 |-- VL_VLR_PARCELA: float (nullable = true)
 |-- DT_INFO: integer (nullable = true)
 |-- ID_PROC: string (nullable = true)

In [49]:
rdd_01.registerTempTable("rdd_01_m")

In [59]:
spark.sql(
             """SELECT 
                    ID_UF,
                    max(VL_VLR_PARCELA) as MAX_VL,
                    min(VL_VLR_PARCELA) as MIN_VL,
                    sum(VL_VLR_PARCELA)/1000000 as SUM_VL_MILHOES,
                    avg(VL_VLR_PARCELA) as AVG_VL,
                    count(*) as QTD_BOLSAS

                FROM
                    rdd_01_m 
                group by 1
                order by count(*) desc
   
             """
         ).show(50)

+-----+------+------+--------------+------------------+----------+
|ID_UF|MAX_VL|MIN_VL|SUM_VL_MILHOES|            AVG_VL|QTD_BOLSAS|
+-----+------+------+--------------+------------------+----------+
|   BA|1464.0|  35.0|    297.931381|   161.57176170594|   1843957|
|   SP|1276.0|  35.0|    208.693312|144.71015229376755|   1442147|
|   PE|1388.0|  35.0|    180.473943|157.79349499665568|   1143735|
|   MG|1310.0|  35.0|     164.85591|149.21125873309384|   1104849|
|   CE|1154.0|  35.0|    170.391273|158.96458098932533|   1071882|
|   MA|1234.0|  35.0|    181.234328|  186.185178070061|    973409|
|   PA|1542.0|  35.0|    163.949987|  178.781328445153|    917042|
|   RJ|1310.0|  35.0|    126.986368|152.03997045079302|    835217|
|   PB|1310.0|  35.0|     90.823443|173.63867051581082|    523060|
|   PI|1388.0|  35.0|     82.982147|182.58380199257186|    454488|
|   RS| 926.0|  35.0|     61.601745|  146.721125814565|    419856|
|   AL|1080.0|  35.0|     68.583884|165.77567649227126|    413

In [61]:
import seaborn as sns
# sns.set(style="whitegrid")
# tips = sns.load_dataset("tips")
# ax = sns.barplot(x="day", y="total_bill", data=tips)

No module named seaborn
Traceback (most recent call last):
ImportError: No module named seaborn

